In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import osmnx as ox
import pandas as pd
import pickle
from pathlib import Path
import os
from tqdm import tqdm
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import requests
import zipfile

#4-step model:
import pandas
import geopandas
import json
import math
from haversine import haversine
from ipfn import ipfn
import networkx
from matplotlib import pyplot
from matplotlib import patheffects


EPSILON = 1e-3

In [2]:
class Agent:
    
    def __init__(self, i, dow, city, alpha=0.5):

        ''' 
        Initialize an Agent instance
        Parameters:
        - i (int): Agent identifier.
        - dow (float): Endowment value.
        - city (City): Reference to the City instance.
        - alpha (float): Weighting factor for transit access vs. community value.
        '''
        self.i = i
        self.dow = dow
        self.city = city 
        self.alpha = alpha

        self.weights = np.ones(len(self.city.centroidDict))
        self.probabilities = np.ones(len(self.city.centroidDict)) # Probability to go to each centroid
        self.tot_probabilities = 0.0
        self.avg_probabilities = None
        self.u = None

        self.reset()
        
    # Create hash identifier
    def __hash__(self):
        return hash(self.i)

    def __eq__(self, other): 
        return self.i == other.i

    '''
    - Assign starting centroid
    - Reset centroid weights/probabilities
    '''
    def reset(self):
        # Normalize probabilities
        self.probabilities[:] = [a/len(self.probabilities) for a in self.probabilities]
        
        self.tot_probabilities = np.sum(self.probabilities)
        
        # Current node - Initialize starting position at random node (based on weights)
        self.u = np.random.choice(list(self.city.centroidDict.keys()), p=self.probabilities) 
        
        # Adds self to node
        self.city.centroidDict[self.u]['inh'].add(self)

# ACTION METHOD
    def act(self): 
        # Leave node
        self.city.centroidDict[self.u]['inh'].remove(self) 
    
        # Choose another node
        self.u = np.random.choice(list(self.city.centroidDict.keys()), p=self.probabilities) 
    
        # Join node
        self.city.centroidDict[self.u]['inh'].add(self) 
        
# LEARN METHOD
    def learn(self):
        for ID, _ in self.city.centroidDict.items():
            self.weights[ID] *= (1 - EPSILON * self.cost(ID)) # Weighted based on COST
        
        self.probabilities = np.array(self.weights / np.sum(self.weights)) # Normalize
        
        for a in self.probabilities:
            self.tot_probabilities += a # used for averaging purposes

# COST FUNCTION
    def cost(self, ID):
        
        # AFFORDABILITY SCORE
        # 1 if self.dow >= node.dow_thr; else 0
        aff = int(self.dow >= self.city.centroidDict[ID]['dow_thr'])
        
        # UPKEEP SCORE
        # 1 if upkeep == True; else 0
        upk = int(self.city.centroidDict[ID]['upk'])
        
        # BELTLINE SCORE
        # 1 if in beltline; else 0
        beltline = int(self.city.centroidDict[ID]['beltline'])
    
        # DISTANCE SCORE
        loc = self.city.centroid_distances[self.u, ID] #very simplistic : multiply cost by normalized distance
        
        # AMENITY ACCESSIBILITY SCORE
        acc = np.exp(- (1 - self.alpha) * self.city.amts_dens[ID])
        
        # COMMUNITY SCORE
        # Difference between node 'cmt' value and self.dow.
        cmt = np.exp(- self.alpha * np.abs(self.dow - self.city.centroidDict[ID]['cmt']))
    
        # COST FUNCTION
        c = 1 - aff * upk * beltline * loc * cmt * acc
        
        return c
        

In [ ]:
# ===============
# FOUR-STEP MODEL
# ===============

# SUPPLY data (transportation network):
url = 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/State_County/MapServer/37/query?where=state%3D06&f=geojson'
r = requests.get(url)
zones = geopandas.GeoDataFrame.from_features(r.json()['features'])
centroidFunction = lambda row: (row['geometry'].centroid.y, row['geometry'].centroid.x)
zones['centroid'] = zones.apply(centroidFunction, axis=1)

# DEMAND data (transportation network users):
url = 'http://api.census.gov/data/2015/acs5/profile?get=NAME,DP03_0018E&for=county&in=state:06'
r = requests.get(url)
Production = pandas.DataFrame(r.json()[1:], columns = r.json()[0], dtype='int')
nameSplit = lambda x: x.split(',')[0]
Production['NAME'] = Production['NAME'].apply(nameSplit)
zones = pandas.merge(zones, Production)
zones['Production'] = zones['DP03_0018E']

def getEmployment(state, county):
    prefix = 'EN'
    seasonal_adjustment = 'U'
    area = format(state, "02d") + format(county, "03d")
    data_type = '1'
    size = '0'
    ownership = '0'
    industry = '10'
    seriesid = prefix + seasonal_adjustment + area + data_type + size + ownership + industry
    headers = {'Content-type': 'application/json'}
    data = json.dumps({"seriesid": [seriesid],"startyear":"2015", "endyear":"2015", "registrationKey": ""})
    p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
    employment = p.json()['Results']['series'][0]['data'][0]['value']
    return(employment)

employment = lambda row: int(getEmployment(row['state'], row['county']))
zones['Attraction'] = zones.transpose().apply(employment)
zones['Production'] = zones['Production'] * zones.sum()['Attraction'] / zones.sum()['Production']
zones.index = zones.NAME
zones.sort_index(inplace=True)

# TRIP DISTRIBUTION:
def costFunction(zones, zone1, zone2, beta):
    cost = math.exp(-beta * haversine(zones[zone1]['centroid'], zones[zone2]['centroid']))
    return(cost)

def costMatrixGenerator(zones, costFunction, beta):
    originList = []
    for originZone in zones:
        destinationList = []
        for destinationZone in zones:
            destinationList.append(costFunction(zones, originZone, destinationZone, beta))
        originList.append(destinationList)
    return(pandas.DataFrame(originList, index=zones.columns, columns=zones.columns))

def tripDistribution(tripGeneration, costMatrix):
    costMatrix['ozone'] = costMatrix.columns
    costMatrix = costMatrix.melt(id_vars=['ozone'])
    costMatrix.columns = ['ozone', 'dzone', 'total']
    production = tripGeneration['Production']
    production.index.name = 'ozone'
    attraction = tripGeneration['Attraction']
    attraction.index.name = 'dzone'
    aggregates = [production, attraction]
    dimensions = [['ozone'], ['dzone']]
    IPF = ipfn.ipfn(costMatrix, aggregates, dimensions)
    trips = IPF.iteration()
    return(trips.pivot(index='ozone', columns='dzone', values='total'))

beta = 0.01
costMatrix = costMatrixGenerator(zones.transpose(), costFunction, beta)
trips = tripDistribution(zones, costMatrix)

# MODE CHOICE:
def modeChoiceFunction(zones, zone1, zone2, modes):
    distance = haversine(zones[zone1]['centroid'], zones[zone2]['centroid'])
    probability = {}
    total = 0.0
    for mode in modes:
        total = total + math.exp(modes[mode] * distance)
    for mode in modes:
        probability[mode] = math.exp(modes[mode] * distance) / total
    return(probability)

def probabilityMatrixGenerator (zones, modeChoiceFunction, modes):
    probabilityMatrix = {}
    for mode in modes:
        originList = []
        for originZone in zones:
            destinationList = []
            for destinationZone in zones:
                destinationList.append(modeChoiceFunction(zones, originZone, destinationZone, modes)[mode])
            originList.append(destinationList)
        probabilityMatrix[mode] = pandas.DataFrame(originList, index=zones.columns, columns=zones.columns)
    return(probabilityMatrix)

modes = {'walking': .05, 'cycling': .05, 'driving': .05}
probabilityMatrix = probabilityMatrixGenerator(zones.transpose(), modeChoiceFunction, modes)
drivingTrips = trips * probabilityMatrix['driving']

#ROUTE ASSIGNMENT:
def routeAssignment(zones, trips):
    G = networkx.Graph()
    G.add_nodes_from(zones.columns)
    for zone1 in zones:
        for zone2 in zones:
            if zones[zone1]['geometry'].touches(zones[zone2]['geometry']):
                G.add_edge(zone1, zone2, distance = haversine(zones[zone1]['centroid'], zones[zone2]['centroid']), volume=0.0)
    for origin in trips:
        for destination in trips:
            path = networkx.shortest_path(G, origin, destination)
            for i in range(len(path) - 1):
                G[path[i]][path[i + 1]]['volume'] = G[path[i]][path[i + 1]]['volume'] + trips[zone1][zone2]
    return(G)

def visualize(G, zones):
    fig = pyplot.figure(1, figsize=(10, 10), dpi=90)
    ax = fig.add_subplot(111)
    zonesT = zones.transpose()
    zonesT.plot(ax = ax)
    for i, row in zones.transpose().iterrows():
        text = pyplot.annotate(s=row['NAME'], xy=((row['centroid'][1], row['centroid'][0])), horizontalalignment='center', fontsize=6)
        text.set_path_effects([patheffects.Stroke(linewidth=3, foreground='white'), patheffects.Normal()])
    for zone1 in G.edge:
        for zone2 in G.edge[zone1]:
            volume = G.edge[zone1][zone2]['volume']
            x = [zones[zone1]['centroid'][1], zones[zone2]['centroid'][1]]
            y = [zones[zone1]['centroid'][0], zones[zone2]['centroid'][0]]
            ax.plot(x, y, color='#444444', linewidth=volume/10000, solid_capstyle='round', zorder=1)
    pyplot.show(block=False)

G = routeAssignment(zones.transpose(), drivingTrips)
visualize(G, zones.transpose())

# TO PLAY AROUND WITH PARAMETERS (LIST OF PARAMETERS:)
# Trip Distribution
    #beta = 0.01
    #costMatrix = costMatrixGenerator(zones.transpose(), costFunction, beta)
    #trips = tripDistribution(zones, costMatrix)
# Mode Choice
    #modes = {'walking': .05, 'cycling': .05, 'driving': .05}
    #probabilityMatrix = probabilityMatrixGenerator(zones.transpose(), modeChoiceFunction, modes)
    #drivingTrips = trips * probabilityMatrix['driving']
# Route Assignment
    #G = routeAssignment(zones.transpose(), drivingTrips)
    #visualize(G, zones.transpose())

In [3]:
class City:

    # CONSTRUCTOR
    def __init__(self, centroids, g, places, rho=2): #default rho (house capacity) == 2
        '''
        Initialize a City instance.
        '''
        self.rho = rho #house capacity
        self.centroid = centroids #centroids list
        self.g = g #OSMnx map
        self.places = places
        
        # Create a dictionary with each centroid's attributes
        self.centroidDict = {} 
        for ID, (lat, lon, name, beltline) in enumerate(self.centroid): #Iterate through each centroid
            self.centroidDict[ID] = {
                'lat': lat, # Latitude
                'lon': lon, # Longitude
                'name': name, # Housing name (region)
                'beltline': beltline,  # Is it in the Beltline?
                
                'inh': set(),  # Set containing all Agent inhabitants
                'dow_thr': 0.0,  # Endowment threshold initialized to 0
                'upk': False,  # Upkeep score
                'cmt': 0.0,  # Community score
                'pop_hist': [], # Population history
                'cmt_hist': [],  # Community history
                
                'node': ox.nearest_nodes(self.g, lon, lat)
            }
        
        #compute density of amenities
        self.amts_dens = self.compute_amts_dens()
        
        # Initialize dictionary of distances between centroids
        self.centroid_distances = self.compute_centroid_distances()
        
    # Distances Between Centroids - helper function
    def compute_centroid_distances(self):
        n = len(self.centroidDict)
        distance_matrix = np.zeros((n, n))
        
        for i in tqdm(range(n), desc="Computing centroid distances"):
            source_node = self.centroidDict[i]['node']
            lengths = nx.single_source_dijkstra_path_length(self.g, source_node, weight='length')
            for j in range(n):
                target_node = self.centroidDict[j]['node']
                distance = lengths.get(target_node, np.inf)
                distance_matrix[i][j] = distance
                    
        if distance_matrix.max() > 0:
            distance_matrix  = distance_matrix/ distance_matrix.max() #normalze

        return distance_matrix

    # Amenity Density (bus stops) - helper function
    def compute_amts_dens(self):
        n = len(self.places)
        areas = np.zeros(n-1)
        bus_stops = np.zeros(n-1)
        tags = {'highway':'bus_stop'} #TODO: I don't think this is accurate
        
        for ID in range(n-1):
            place = self.places[ID+1]
            print(place)
            
            # (AREA): Get the boundary polygon of the graph
            gdf = ox.geocode_to_gdf(place)
            # Project the GDF to a suitable CRS for area calculations
            gdf_projected = ox.project_gdf(gdf)
            # Calculate the area
            area = gdf_projected.area.iloc[0]
            areas[ID] = area
            
            # (AMENITIES): Get polygon
            polygon = gdf['geometry'].union_all()
            
            try:
                # Get bus stops within the polygon
                gdf_bus = ox.features_from_polygon(polygon, tags)
                if gdf_bus.empty:
                    bus_stops[ID] = 0
                else:
                    bus_stops[ID] = len(gdf_bus)
            except:
                print(f"No data elements returned for {place}. Setting bus stops to 0.")
                bus_stops[ID] = 0
        
    
        # Calculate density in bus stops per square km
        amts_dens = (bus_stops / areas) * 1e6  # Multiply by 1e6 to convert sq meters to sq km
        if max(amts_dens) > 0:
            amts_dens = amts_dens/max(amts_dens) # Normalize
        print ("Areas:")
        print (areas)
        print ("Number of bus stops:")
        print(bus_stops)
        print ("Normalizd bus stop densities:")
        print (amts_dens)
        return amts_dens
            

    # Set agents and their endowments
    def set_agts(self, agts):
        self.agts = agts #list of agents
        self.agt_dows = np.array([a.dow for a in self.agts]) #array of agent endowments

    # Update each node
    def update(self):   
        for ID, data in self.centroidDict.items(): # For each centroid

            pop = len(data['inh']) # Inhabitants
            inhabitant_dows = [a.dow for a in data['inh']]  # Array of endowments of node's inhabitants
            
            # COMMUNITY SCORE (average endowment)
            cmt = 0.0
            if pop > 0:
                distances = self.centroid_distances[ID, [agent.u for agent in data['inh']]]
                weights = (1 - distances) ** 2
                
                if np.sum(weights) > 0:
                    cmt = np.average(inhabitant_dows, weights=weights)
                data['cmt'] = cmt
            
            # UPKEEP SCORE
            # ENDOWMENT THRESHOLD
            if pop > 0: # If inhabited
                if pop < self.rho:
                    data['dow_thr'] = 0.0
                else:
                    data['dow_thr'] = np.partition(inhabitant_dows, -self.rho)[-self.rho] # Lowest endowment value if Population = Rho
                data['upk'] = True
                
            else: # If uninhabited
                data['dow_thr'] = 0.0
                data['upk'] = False

            # Update population history
            data['pop_hist'].append(pop)
            # Update Community history (average endowment)
            data['cmt_hist'].append(cmt)
        
        
    # =============
    # PLOTTING CODE
    # =============
    def plot(self, cmap='YlOrRd', figkey='city', graph=None):
        fig, ax = plt.subplots(figsize=(10, 10))
    
        if graph:
            ox.plot_graph(graph, ax=ax, node_color='black', node_size=10, edge_color='gray', edge_linewidth=1, show=False, close=False)
    
        # Prepare agent data
        agent_lats = [self.centroidDict[agent.u]['lat'] for agent in self.agts]
        agent_lons = [self.centroidDict[agent.u]['lon'] for agent in self.agts]
        agent_wealths = [agent.dow for agent in self.agts]
    
        # Population density heatmap
        heatmap, xedges, yedges = np.histogram2d(agent_lons, agent_lats, bins=30)
        extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
        ax.imshow(heatmap.T, extent=extent, origin='lower', cmap=cmap, alpha=0.5)
    
        # Plot agents with wealth-based marker sizes
        norm = Normalize(vmin=min(agent_wealths), vmax=max(agent_wealths))
        marker_sizes = [50 + 150 * norm(w) for w in agent_wealths]
        sc = ax.scatter(agent_lons, agent_lats, c=agent_wealths, s=marker_sizes, cmap='coolwarm', alpha=0.7, edgecolor='red')
    
        # Plot centroids locations (this comes after the graph to make sure they are visible on top)
        for ID, house in self.centroidDict.items():
            lat, lon = house['lat'], house['lon']
            color = 'yellow' if house['beltline'] else 'white'
            ax.scatter(lon, lat, color=color, s=100, alpha=0.7, edgecolor='black')
            
            # Display inhabitant populations at each node:
            inhabitants = len(house['inh'])
            ax.text(lon, lat, str(inhabitants), fontsize=9, ha='center', va='center', color='black')

    
        # Add color bar for wealth
        cbar = plt.colorbar(sc, ax=ax, orientation='vertical', label='Wealth (dow)')
    
        # Labels and title
        ax.set_title(f"City Visualization: {figkey}")
        ax.set_xlabel("Longitude")
        ax.set_ylabel("Latitude")
    
        # Legend
        ax.scatter([], [], c='yellow', s=100, label='Beltline Housing')
        ax.scatter([], [], c='white', s=100, label='Non-Beltline Housing')
        ax.scatter([], [], c='red', s=100, label='Agents')
        ax.legend(loc='upper right')
    
        plt.tight_layout()
        plt.savefig(f'./figures/{figkey}.pdf', format='pdf', bbox_inches='tight')
        plt.close()

In [4]:
def download_and_extract_file(url, filename):
    # Use a 'data' subfolder in the current working directory
    cwd = Path.cwd()
    data_dir = cwd / "data"
    data_dir.mkdir(exist_ok=True)  # Create the 'data' directory if it doesn't exist
    file_path = data_dir / filename

    # Create extraction subfolder name (remove .zip extension)
    extract_folder_name = filename.rsplit('.', 1)[0]
    extract_path = data_dir / extract_folder_name
    
    # Check if file already exists
    if file_path.exists():
        print(f"{filename} already exists in {data_dir}. Skipping download.")
    else:
        # Make the request
        print(f"Downloading {filename} to {data_dir}...")
        response = requests.get(url, stream=True)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Get the total file size
            total_size = int(response.headers.get('content-length', 0))

            # Open the file and use tqdm for the progress bar
            with file_path.open('wb') as file, tqdm(
                desc=filename,
                total=total_size,
                unit='iB',
                unit_scale=True,
                unit_divisor=1024,
            ) as progress_bar:
                for data in response.iter_content(chunk_size=1024):
                    size = file.write(data)
                    progress_bar.update(size)
            print(f"Successfully downloaded {filename}")
        else:
            print(f"Failed to download {filename}. Status code: {response.status_code}")
            return

    # Extract the ZIP file
    print(f"Extracting {filename} to {extract_path}...")
    extract_path.mkdir(exist_ok=True)  # Create the extraction folder if it doesn't exist
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        # Get the total number of files in the ZIP
        total_files = len(zip_ref.infolist())
        
        # Use tqdm for the extraction progress bar
        for file in tqdm(zip_ref.infolist(), desc="Extracting", total=total_files):
            zip_ref.extract(file, extract_path)
    
    print(f"Successfully extracted {filename} to {extract_path}")

# URL of the file to download
url = "https://www2.census.gov/geo/tiger/TIGER2022/ZCTA520/tl_2022_us_zcta520.zip"

# Filename to save as
filename = "tl_2022_us_zcta520.zip"

# Call the function to download and extract the file
download_and_extract_file(url, filename)

tl_2022_us_zcta520.zip already exists in c:\Users\kmmat\OneDrive\Desktop\VIP\24Fa-MPONC\modeling_processes_of_neighborhood_change_new\data. Skipping download.
Extracting tl_2022_us_zcta520.zip to c:\Users\kmmat\OneDrive\Desktop\VIP\24Fa-MPONC\modeling_processes_of_neighborhood_change_new\data\tl_2022_us_zcta520...


Extracting: 100%|██████████| 7/7 [00:03<00:00,  1.84it/s]

Successfully extracted tl_2022_us_zcta520.zip to c:\Users\kmmat\OneDrive\Desktop\VIP\24Fa-MPONC\modeling_processes_of_neighborhood_change_new\data\tl_2022_us_zcta520


In [5]:
cwd = Path.cwd()
print(cwd)
figures_folder = Path(cwd / "figures")
if not os.path.isdir(figures_folder):
    os.makedirs(figures_folder)


# Centroids
'''centroids = [
    (33.7501, 84.3885, 'RDA/Cascade', True),
    (33.7501, 84.3885, 'Pittsburgh/Peoplestown', True),
    (33.7501, 84.3885, 'Boulevard Crossing', True),
    (33.73586185,-84.3709322239104, 'Memorial Drive/Glenwood/Grant Park', True),
    (33.7680818,-84.36505111969021, 'Freedom Parkway/Fourth Ward Park', True),
    (33.7501, 84.3885, 'Virginia Highlands/Ansley/Piedmont Park', True),
    (0.0, 0.0, 'Peachtree/Collier', True),
    (0.0, 0.0, 'Upper Westside/Northside', True),
    (0.0, 0.0, 'Simpson/Hollowell', True),
    (33.779241999999996,-84.43839416079305, 'Upper Marietta/Westside Park', True)'''
centroids = [
    (33.6534427,-84.4493725, 'College Park', False),
    (33.6795531,-84.4393724, 'East Point', False),
    (33.9242688,-84.3785379, 'Sandy Springs', True),
    (33.7737582,-84.296069, 'Decatur', False),
]

# (GRAPH APPROACH - automate creation of nodes) 
# Getting nodes with OSMnx
load_g = False
if not load_g:
    '''gdf = gpd.read_file(cwd / Path('data/tl_2022_us_zcta520/tl_2022_us_zcta520.shp'))
    gdf = gdf[gdf['ZCTA5CE20'] == '11206']
    shape = gdf.iloc[0].geometry'''
    #IF LOADING SPECIFIC SHAPEFILE(S)
   
    places = [
    'Atlanta, GA',
    'College Park, Fulton County, GA, USA',
    'East Point, GA',
    'Sandy Springs, GA',
    'Decatur, DeKalb County, GA',
    ]
    g = ox.graph_from_place(places, network_type='drive', simplify=True)
    #Roadmap of Atlanta
    g = g.subgraph(max(nx.strongly_connected_components(g), key=len)).copy()
    #Ensures all nodes are connected
    g = nx.convert_node_labels_to_integers(g)
    #Converts nodes to integers
   
    with open(Path(cwd / 'data/tl_2022_us_zcta520/atlanta.pkl'), 'wb') as file:
        pickle.dump(g, file)
else:
    with open(Path(cwd / 'data/tl_2022_us_zcta520/atlanta.pkl'), 'rb') as file:
        g = pickle.load(file)


# ====================================
# SIMULATION PRE-DETERMINED PARAMETERS
# ====================================
rho_l = [2] #1, 2, 4, 8 (for each iteration) rho-house capacity
alpha_l = [0.25] #0.25, 0.75 (for each iteration) lambda - centroid proximity vs. community value
t_max_l = [10] #5000, 10000, 15000, 20000 (for each iteration) timesteps
tau = 0.5 # inequality factor in Lorentz curve
num_agents = 50

# RUN SIMULATION?
run_experiments = True

# PLOT SIMULATION?
plot_cities = True

cty_key = 'Atlanta'


# ===============
# SIMULATION CODE
# ===============

if run_experiments:
    for rho in rho_l:
        for alpha in alpha_l:

            np.random.seed(0)

            city = City(centroids, g, places, rho=rho)
            agt_dows = np.diff([1 - (1 - x) ** tau for x in np.linspace(0, 1, num_agents + 1)]) 
            agts = [Agent(i, dow, city, alpha=alpha) for i, dow in enumerate(agt_dows)]

            city.set_agts(agts)
            city.update()

            for t in range(max(t_max_l)):
                print('t: {0}'.format(t))
                for a in agts:
                    a.act()
                city.update()
                for a in agts:
                    a.learn()
                
                if t + 1 in t_max_l:

                    for a in city.agts:
                        a.avg_probabilities = a.tot_probabilities / (t + 1)

                    with open(Path(cwd / 'data/{0}_{1}_{2}_{3}.pkl'.format(cty_key, rho, alpha, t + 1)), 'wb') as file:
                        pickle.dump(city, file)

if plot_cities:
    for rho in rho_l:
        for alpha in alpha_l:
            for t_max in t_max_l:
                with open(Path(cwd / 'data/{0}_{1}_{2}_{3}.pkl'.format(cty_key, rho, alpha, t_max)), 'rb') as file:
                    city = pickle.load(file)
                cmap = 'YlOrRd'
                figkey = '{0}_{1}_{2}_{3}'.format(cty_key, rho, alpha, t_max)
                city.plot(cmap=cmap, figkey=figkey, graph=g)

c:\Users\kmmat\OneDrive\Desktop\VIP\24Fa-MPONC\modeling_processes_of_neighborhood_change_new
College Park, Fulton County, GA, USA
East Point, GA
Sandy Springs, GA
Decatur, DeKalb County, GA
Areas:
[27975135.28790003 38139324.82998758 99831022.65713052 11498367.60443545]
Number of bus stops:
[17. 13.  3. 16.]
Normalizd bus stop densities:
[0.43670979 0.24495514 0.02159593 1.        ]


Computing centroid distances: 100%|██████████| 4/4 [00:00<00:00, 18.00it/s]


t: 0
t: 1
t: 2
t: 3
t: 4
t: 5
t: 6
t: 7
t: 8
t: 9
